# Neural networks code and tests notebook

# This notebook takes the modified raw data and try to predict classes on  :

# - Conv1D network

In [ ]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame


In [ ]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF_GOLD/OFF_data/"

In [ ]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [ ]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)

        if lg==binning:
            idx_ON.append(i)

    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [ ]:
Feature_time_series=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

In [ ]:


lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Class_ON = []

nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))

#Construct data matrix
for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j])
    dataframe.columns=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['fratio']
    bigdata[2][j]=dataframe['Delta Flux']*dataframe['Flux']
    bigdata[3][j]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[4][j]=dataframe['fratio']*dataframe['Photon Index']
    bigdata[5][j]=dataframe['Photon Index']
    bigdata[6][j]=abs(dataframe['Photon Index'])-dataframe['fratio']
    bigdata[7][j]=np.var(dataframe['Photon Index'])*dataframe['Photon Index'] 
    bigdata[8][j]=np.var(dataframe['fratio'])*dataframe['fratio'] 

for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j])
    dataframe.columns=['Iteration','MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][v]=dataframe['Flux']
    bigdata[1][v]=dataframe['fratio']  
    bigdata[2][v]=dataframe['Delta Flux']*dataframe['Flux']
    bigdata[3][v]=dataframe['Photon Index']*dataframe['Flux']
    bigdata[4][v]=dataframe['fratio']*dataframe['Photon Index']
    bigdata[5][v]=dataframe['Photon Index']
    bigdata[6][v]=abs(dataframe['Photon Index'])-dataframe['fratio']
    bigdata[7][v]=np.var(dataframe['Photon Index'])*dataframe['Photon Index']
    bigdata[8][v]=np.var(dataframe['fratio'])*dataframe['fratio'] 
    


#Creating labels
for i in range(lgOFF):
    
    Class_ON.append(int(0))
    
for i in range(lgON):
    Class_ON.append(int(1))


In [ ]:
#Reshaping data matrix
bigdata=bigdata.reshape(c,nbfeatures,binning)

In [ ]:
# Conv1D architecture
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Conv1D

csv_logger = CSVLogger(model_history4, append=True)
weight_for_0 = (1 / lgOFF) * (lg / 2.0)
weight_for_1 = (1 / lgON) * (lg / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
bigdata=bigdata.reshape(c,nbfeatures,binning)
nb_filters = 20
fsize=1
x_train, x_test, y_train, y_test = train_test_split(bigdata, Class_ON, test_size=0.3, random_state=38)
y_test2=y_test.copy()
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



model = Sequential()
model.add(Conv1D(nb_filters, 1, padding="same", activation="relu",input_shape=(nbfeatures,binning)))
model.add(Conv1D(15, 1, padding="same", activation="relu",input_shape=(nb_filters,binning)))
model.add(Conv1D(9, 1, padding="same", activation="relu",input_shape=(15,binning)))
model.add(GlobalMaxPooling1D())
model.add(Dense(binning, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train,epochs=160,batch_size=4,class_weight=class_weight)


In [ ]:
ON_accuracy=[]
OFF_accuracy=[]

#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(1,35):
    
    #Conv1D Architecture
    model = Sequential()
    model.add(Conv1D(nb_filters, 1, padding="same", activation="relu",input_shape=(nbfeatures,binning)))
    model.add(Conv1D(15, 1, padding="same", activation="relu",input_shape=(nb_filters,binning)))
    model.add(Conv1D(9, 1, padding="same", activation="relu",input_shape=(15,binning)))

    model.add(GlobalMaxPooling1D())

    model.add(Dense(binning, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(bigdata, Class_ON, test_size=0.33, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=70,batch_size=4,class_weight=class_weight)

    #Obtain the accuradcy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0

    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")